In [124]:
from collections import defaultdict, namedtuple
from importlib import reload
from pathlib import Path
from string import ascii_lowercase as LETTERS
from random import sample, choice, uniform
from time import time
from math import log, exp
import codecs
import csv
import random
import sys
import urllib.request
import zipfile

from tqdm import tqdm
from sklearn.datasets import fetch_20newsgroups
import numpy as np
import pandas as pd

sys.path.insert(0, str(Path.cwd().parents[0] / 'src')) 
import crypto as cg; reload(cg)

<module 'crypto' from '/Users/chris/Documents/cryptogram-solver/src/crypto.py'>

In [13]:
'https://www.kaggle.com/snapcrack/all-the-news/version/4'
article_num = 1
filename = f'/Users/chris/Downloads/articles{article_num}.csv.zip'
archive = zipfile.ZipFile(filename, 'r')

In [14]:
tmp = archive.read('articles1.csv')

In [63]:
def get_documents():
    csv.field_size_limit(sys.maxsize)
    articles = list()
    max_rows = 250
    for article_num in range(1, 4):
        zipname = Path('..', 'data', 'raw', f'articles{article_num}.csv.zip')
        filename = f'articles{article_num}.csv'
        with zipfile.ZipFile(zipname) as z:
            with z.open(filename, 'r') as f:
                csvfile = csv.reader(codecs.iterdecode(f, 'utf-8'))
                for row, line in enumerate(csvfile):
                    articles.append(line[-1])  # get content column
                    if row == max_rows:
                        break
    articles = articles[1:]  # remove column header
    return articles

In [64]:
docs = get_documents()

In [65]:
# news_train = fetch_20newsgroups(subset='train', shuffle=True, random_state=20180827)
# news_test = fetch_20newsgroups(subset='test', shuffle=True, random_state=20180827)

In [125]:
tokenizer_test = cg.Tokenizer()
tokenizer_test.tokenize("This is a sentence.")

{Token(ngrams=('this',), kind=<NgramKind: word>, n=1): 1,
 Token(ngrams=('is',), kind=<NgramKind: word>, n=1): 1,
 Token(ngrams=('a',), kind=<NgramKind: word>, n=1): 1,
 Token(ngrams=('sentence',), kind=<NgramKind: word>, n=1): 1,
 Token(ngrams=('<',), kind=<NgramKind: char>, n=1): 4,
 Token(ngrams=('t',), kind=<NgramKind: char>, n=1): 2,
 Token(ngrams=('h',), kind=<NgramKind: char>, n=1): 1,
 Token(ngrams=('i',), kind=<NgramKind: char>, n=1): 2,
 Token(ngrams=('s',), kind=<NgramKind: char>, n=1): 3,
 Token(ngrams=('>',), kind=<NgramKind: char>, n=1): 4,
 Token(ngrams=('<', 't'), kind=<NgramKind: char>, n=2): 1,
 Token(ngrams=('t', 'h'), kind=<NgramKind: char>, n=2): 1,
 Token(ngrams=('h', 'i'), kind=<NgramKind: char>, n=2): 1,
 Token(ngrams=('i', 's'), kind=<NgramKind: char>, n=2): 2,
 Token(ngrams=('s', '>'), kind=<NgramKind: char>, n=2): 2,
 Token(ngrams=('<', 't', 'h'), kind=<NgramKind: char>, n=3): 1,
 Token(ngrams=('t', 'h', 'i'), kind=<NgramKind: char>, n=3): 1,
 Token(ngrams=('

In [126]:
reload(cg)
tokenizer = cg.Tokenizer(char_ngram_range=(1, 3), word_ngram_range=(1, 1), vocab_size=1000000)
# tokenizer.fit(news_train['data'][:5000])
tokenizer.fit(docs)
print(len(tokenizer.vocab))
print(tokenizer.totals)

100%|██████████| 752/752 [00:32<00:00, 23.25it/s]

42219
{(<NgramKind: word>, 1): 851576, (<NgramKind: char>, 1): 5733872, (<NgramKind: char>, 2): 4882296, (<NgramKind: char>, 3): 4030720}


In [129]:
tokenizer.totals

{(<NgramKind: word>, 1): 851576,
 (<NgramKind: char>, 1): 5733872,
 (<NgramKind: char>, 2): 4882296,
 (<NgramKind: char>, 3): 4030720}

In [130]:
doc = cg.Doc(articles[0].lower())
mapper = cg.Mapping()
mapper.scramble()
doc = mapper.translate(doc)
doc

'mnqrjaecha  —   vhaefyqqjhant fygdktjvnaq rnpy n aym synf mrya jc vhlyq ch cryjf    ryntcr vnfy tnmqdjc nenjaqc cry hknln nxljajqcfncjha: cryz ljerc mja. cry javhljae cfdlg nxljajqcfncjha vhdtx vrhhqy ch ah thaeyf xysyax cry yoyvdcjpy kfnavr nenjaqc cry qdjc, mrjvr vrnttyaeyq cry nxljajqcfncjha’q ndcrhfjcz ch qgyax kjttjhaq hs xhttnfq ha ryntcr jaqdfnavy qdkqjxjyq shf   nax   nlyfjvnaq, rnaxjae rhdqy fygdktjvnaq n kje pjvchfz ha    jqqdyq. kdc n qdxxya thqq hs cry xjqgdcyx qdkqjxjyq vhdtx vhavyjpnktz vndqy cry ryntcr vnfy gfhefnl ch jlgthxy, tynpjae ljttjhaq hs gyhgty mjcrhdc nvvyqq ch ryntcr jaqdfnavy kyshfy fygdktjvnaq rnpy gfygnfyx n fygtnvylyac. crnc vhdtx tynx ch vrnhq ja cry jaqdfnavy lnfbyc nax qgdf n ghtjcjvnt knvbtnqr wdqc nq fygdktjvnaq enja sdtt vhacfht hs cry ehpyfalyac. ch qcnpy hss crnc hdcvhly, fygdktjvnaq vhdtx sjax crylqytpyq ja cry nmbmnfx ghqjcjha hs nggfhgfjncjae rdey qdlq ch cylghfnfjtz gfhg dg cry hknln ryntcr vnfy tnm, naeyfjae vhaqyfpncjpy phcyfq mrh rnpy kyya 

In [131]:
[print(f'{x:0.3g}') for x in np.exp(np.linspace(3, -4, 5))];

20.1
3.49
0.607
0.105
0.0183


In [165]:
'''
Ideas:
> Implement simulated annealing properly (single swap and decision), with a reasonable temp scheduler.
  - This works! But need to play with temperature scheduler since results are sensitive to even small changes.
> Implement simulated annealing but using softmax and subsetting swap options.
> More swaps in the beginning, fewer later.
'''

solver = cg.Solver(tokenizer, None, 0.1)

def simulated_annealing(text, solver, tokenizer, max_epochs=10000, is_debug=True):
    best_mapping = mapping = cg.Mapping()
    doc = cg.Doc(text)
    best_score = solver.score(doc)
    epoch = 0
    decisions = defaultdict(int)
    temps = np.exp(np.linspace(0, -6, 10000))
    decisions['good'], decisions['bad_keep'], decisions['bad_pass'] = 0, 0, 0
    for temp in tqdm(temps):
        # print('*', end='')
        improving = False
        new_mapping = mapping.random_swap(doc.letters)
        new_doc = new_mapping.translate(doc)
        score = solver.score(new_doc)
        score_change = score - best_score
        # if score_change < 0 or exp(-score_change / temp) > uniform(0, 1):
        #     print(f'score change: {score_change}')
        #     print('updating')
        #     best_mapping = mapping
        #     best_score = score
        if score_change < 0:
            # print('updating (improvement)')
            best_mapping = new_mapping
            best_score = score
            decisions['good'] += 1
        elif exp(-score_change / temp) > uniform(0, 1):
            # Break this out as different section just for debugging.
            # print(f'updating (bad change): {score_change}')
            best_mapping = new_mapping
            best_score = score
            decisions['bad_keep'] += 1
        else:
            decisions['bad_pass'] += 1
            # print(f'keeping: {score_change}')
        mapping = best_mapping
        epoch += 1
        if epoch % 1000 == 0:
            print(f'{score:0.5g}, {mapping.mapping}, {mapping.translate(doc).text}')
            # print(sorted(list(decisions.items())))
            print(pd.DataFrame(sorted(list(decisions.items()))))
            decisions = defaultdict(int)
    print(f'\nfinal best ({epoch} epochs): {best_score:0.5g}')
    return mapping.translate(doc).text

In [173]:
import spacy
nlp = spacy.load('en')

In [181]:
# text = doc.text
# text = 'Rbo rpktigo vcrb bwucja wj kloj hcjd, km sktpqo, cq rbwr loklgo vcgg cjqcqr kj skhcja wgkja wjd rpycja rk ltr rbcjaq cj cr. -- Roppy Lpwrsborr'
# text = 'This is the story of a girl. Who cried a river and drowned the whole world. And while she looked so sad in photographs, I absolutely love her when she smiles.'
# text = 'Jail zesty vixen who grabbed pay from quack.'
# text = "I've found that when everyone rallies behind a cause, and when they learn their effort can contribute something bigger, they get engaged."
text = str(random.choice(list(nlp(random.choice(articles)).sents)))
doc = cg.Doc(text.lower())
mapping = cg.Mapping()
mapping.scramble()
doc = mapping.translate(doc)
doc.text

'ewi zmiil, rwx kqhilfif ex ewi ewaxli nl 1952, gihkvi ewi rxacf’q   vxlkahw jxccxrnls ewi fikew xj dnls gwmvngxc kfmctkfib xj ewkncklf nl xhexgia.'

In [182]:
text = simulated_annealing(doc.text, solver, tokenizer)
print(text)

 12%|█▏        | 1167/10000 [00:01<00:10, 828.28it/s]

17.154, yegrxcnuilqkhjftozwsvdmpba, bsi rwiij, dse lkmijoio be bsi bszeji gj 1952, cimlui bsi dezfo’k   uejlzms neffedgjt bsi oilbs en vgjt cswugcef lowfploiy en bslgfljo gj embeciz.
          0    1
0  bad_keep  415
1  bad_pass  185
2      good  400


 21%|██        | 2099/10000 [00:02<00:09, 819.74it/s]

13.876, evcnlpuoxbharyqfjimkwzsdtg, aur vsrre, mui tokreprp ai aur aulier de 1952, zrktbr aur milcp’o   bietlku qiccimdew aur prtau iq xdew zusbdzic tpscytprj iq autdctep de ikaizrl.
          0    1
0  bad_keep  348
1  bad_pass  293
2      good  359


 31%|███       | 3104/10000 [00:03<00:08, 820.07it/s]

...

10.775, kghfijswnydcvlxzuaqemtrbop, the pueen, who ascended to the throne in 1952, became the world’s   monarch following the death of king bhumibol adulvadex of thailand in october.
          0    1
0  bad_keep    3
1  bad_pass  995
2      good    2


 91%|█████████ | 9120/10000 [00:11<00:01, 812.72it/s]

11.287, kghfijswnydcvlxtzaqemurbop, the queen, who ascended to the throne in 1952, became the world’s   monarch following the death of king bhumibol adulpadex of thailand in october.
          0    1
0  bad_pass  998
1      good    2


100%|██████████| 10000/10000 [00:12<00:00, 812.81it/s]

9.8184, kghfijswnydcvlxbzaqemurotp, the queen, who ascended to the throne in 1952, became the world’s   monarch following the death of king bhumibol adulyadep of thailand in october.
          0    1
0  bad_pass  998
1      good    2

final best (10000 epochs): 9.5236
the queen, who ascended to the throne in 1952, became the world’s   monarch following the death of king bhumibol adulyadep of thailand in october.
